# Classes, Dataclasses, Pydantic classes

| Feature               | Regular Class | `@dataclass` | Pydantic Model    |
| --------------------- | ------------- | ------------ | ----------------- |
| Built-in Type Hints   | ✅            | ✅            | ✅                 |
| Boilerplate Reduction | ❌            | ✅            | ✅                 |
| Type Coercion (e.g., str→float) | ❌             | ❌                              | ✅                   |
| Runtime Type Checking | ❌            | ❌            | ✅                 |
| Custom Validation     | ❌            | Limited      | ✅ (`@field_validator`)  |
| Auto Serialization    | ❌            | ❌            | ✅ (`.json()`)     |
| Error Messages           | ❌             | ❌                              | ✅                   |
| Standard Library      | ✅            | ✅            | ❌ (pip install) |




#### Class with basic capabilities of dataclass

In [ ]:
class GeoPoint:
    def __init__(self, longitude: float, latitude: float):
        self.longitude = longitude
        self.latitude = latitude

    def __repr__(self):
        return f"GeoPoint(longitude={self.longitude}, latitude={self.latitude})"



longitude, latitude = -75.7552, 45.3484
pnt = GeoPoint(longitude=longitude, latitude=latitude)
print(pnt)


#### @dataclass

In [ ]:
from dataclasses import dataclass

@dataclass
class GeoPoint:
    longitude: float
    latitude: float



longitude, latitude = -75.7552, 45.3484
pnt = GeoPoint(longitude=longitude, latitude=latitude)
print(pnt)

#### Pydantic class

In [ ]:
from pydantic import BaseModel

class GeoPoint(BaseModel):
    longitude: float
    latitude: float



longitude, latitude = -75.7552, 45.3484
pnt = GeoPoint(longitude=longitude, latitude=latitude)
print(pnt)
print(repr(pnt))

#### Type coercion

In [ ]:
class GeoPoint:
    def __init__(self, longitude: float, latitude: float):
        self.longitude = longitude
        self.latitude = latitude


pnt = GeoPoint(longitude="-75", latitude=45)
print(f"Python class type(pnt.longitude)   : {type(pnt.longitude)}")



from pydantic import BaseModel

class GeoPoint(BaseModel):
    longitude: float
    latitude: float


pnt = GeoPoint(longitude="-75", latitude=45)
print(f"Pydantic class type(pnt.longitude) : {type(pnt.longitude)}")

##### Runtime Type Checking

In [ ]:
from pydantic import BaseModel, Field, ValidationError

class GeoPoint(BaseModel):
    longitude: float = Field(ge=-180, le=180)
    latitude: float = Field(ge=0, le=90)


longitude, latitude = 200, 500
try:
    pnt = GeoPoint(longitude=longitude,
                   latitude=latitude)
    print(pnt)
except ValidationError as e:
    print(e)


##### Custom validation

In [31]:
from pydantic import BaseModel, model_validator, ValidationError

class GeoPoint(BaseModel):
    latitude: float | None = None
    longitude: float | None = None

    @model_validator(mode="after")
    def validate_pair(self):
        if (self.latitude is None) != (self.longitude is None):
            raise ValueError(
                "latitude and longitude must both be set or both be None"
            )
        return self


try:
    pnt = GeoPoint(longitude=None, latitude=45.0)
    print(pnt)
except ValidationError as e:
    print(e)


1 validation error for GeoPoint
  Value error, latitude and longitude must both be set or both be None [type=value_error, input_value={'longitude': None, 'latitude': 45.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


##### Auto-serialization

In [34]:
from pydantic import BaseModel

class GeoPoint(BaseModel):
    latitude: float | None = None
    longitude: float | None = None
    
    
pnt = GeoPoint(longitude=-75, latitude=45)
json_str = pnt.model_dump_json()
print("Model to JSON:")
print(json_str)

print("JSON to model:")
pnt = GeoPoint.model_validate_json(json_str)
print(pnt)


Model to JSON:
{"latitude":45.0,"longitude":-75.0}
JSON to model:
latitude=45.0 longitude=-75.0
